### 프로젝트: 포켓몬 이미지를 수집
- 포켓몬 공식 사이트에서 제공하는 포켓몬 도감의 이미지를 저장

In [2]:
import os

In [8]:
# 폴더 생성
# - 기존에 중복 폴더가 존재하면 ERROR
path = "pokemon" # 폴더 경로
if not os.path.isdir(path):
    os.mkdir(path)

In [92]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd
import time
from tqdm import tqdm as tq
from bs4 import BeautifulSoup as bs

In [94]:
options = Options()
options.add_argument("disable-blink-features=AutomationControlled")  # 자동화 탐지 방지
options.add_experimental_option("excludeSwitches", ["enable-automation"])  # 자동화 표시 제거
options.add_experimental_option('useAutomationExtension', False)  # 자동화 확장 기능 사용 안 함

driver_path = ChromeDriverManager().install()
correct_driver_path = os.path.join(os.path.dirname(driver_path), "chromedriver.exe")
driver = wb.Chrome(service=Service(executable_path=correct_driver_path), options=options)

In [96]:
driver.get("https://pokemonkorea.co.kr/pokedex")

##### 정적 크롤링
- requests.text, driver.page_source → 정적(문자열 → 파싱(BeautifulSoup))

##### 동적 크롤링
- driver.find_elements() -> 현재 보고 있는 웹페이지가 변경되면, 요소 지정도 변경 해야함!

In [98]:
# 현재 selenium 웹 브라우저가 접속하고 있는 페이지의 전체 HTML 소스를 가져와서
# BeautifulSoup을 사용해서 -> parsing
soup = bs(driver.page_source, "lxml")

In [100]:
img = soup.select("img.img-fluid")
print(len(img))

18


In [102]:
# URL 내용을 파일로 다운로드 가능!
from urllib.request import urlretrieve

# urlretrieve(img[0]["src"], "pokemon/pokemon_1.jpeg")

In [35]:
# 포켓몬 18마리 이미지 저장 -> pokemon
# 이미지 이름은 pokemon_1.jpg
for i, pokemon in enumerate(img):
    # 이미지 저장
    urlretrieve(pokemon["src"], f"pokemon/pokemon_{i+1}.jpeg")

#### Selenium 스크롤 제어!
- 마우스 휠 스크롤 제어 존재하지 않음!
- 키보드 입력으로 스크롤 내리기 가능!

In [104]:
body = driver.find_element(By.TAG_NAME, "body")

# 화면 최대화
driver.maximize_window()

In [106]:
# 스크롤 하강 알고리즘
# body.send_keys(Keys.END)

#스크롤 내리기 전의 길이 저장
old_height = driver.execute_script("return document.body.scrollHeight")

In [90]:
while True: #break문을 함께 사용
    #1. body태그 가져와서 스크롤 내리기!
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time_sleep(1)

    #2. 스크롤 내리고 난 후 길이 저장
    new_height=driver.execute_script("return document.body.scrollHeight")

    #3. 스크롤 내리기 전과 후의 길이가 같으면 멈춤(마지막 스크롤)
    if old_height == new_height:
        break

    #4. 이전 페이지의 높이를 새로운 페이지의 높이로 업데이트
    old_height=new_height
    
driver.close()

76347

### 프로젝트: 포켓몬(1025마리) 도감 수집!

In [ ]:
# 1. 수집한 포켓몬 도감을 저장할 폴더 생성!
path = "pokemon"
if not os.path.isdir(path):
    os.mkdir(path)

# 2. 웹 브라우저 On 및 포켓몬 사이트 접속
driver_path = ChromeDriverManager().install()
correct_driver_path = os.path.join(os.path.dirname(driver_path), "chromedriver.exe")
driver = wb.Chrome(service=Service(executable_path=correct_driver_path), options=options)
driver.get("https://pokemonkorea.co.kr/pokedex")

#3. 스크롤 하강
driver.maximize_window()  # 화면 최대화
old_height = driver.execute_script("return document.body.scrollHeight")  #스크롤 내리기 전의 길이 저장
while True: #break문을 함께 사용
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.END)
    time_sleep(1)
    new_height=driver.execute_script("return document.body.scrollHeight")
    if old_height == new_height:
        break
    old_height=new_height

#4. 이미지 수집 및 저장
img = driver.find_elements(By.CSS_SELECTOR, "img.img-fluid")
for i in tq(range(len(img))):
    ultretrieve(img[i].get_attribute(src), f"pokemon/pokemon_{i+1}.jpg")

# 5. 종료
driver.quit()

### 심화: 해당 요소 wait 기법
- time.sleep() -> 고정적으로 ?초동안 기다렸다가 실행!
- 페이지 로딩 시간은 시시각각 다름(인터넷 속도, 서버의 트래픽, 부하 등의 요소 영향을 받기 때문)
- wait 모듈을 활용하면, 좀 더 효율적으로 페이지 load를 기다려 볼 수 있음

#### SEO(검색 엔진 최적화)
    - 웹사이트가 검색 결과에 더 잘 보이도록 최적화하는 과정<br>

#### SSR(Server Side Rendering)
    - 클라이언트 -> 서버 -> 서버: 랜더링(그림) -> 클라이언트<br>
    - 서버 측에서 렌더링해서 클라이언트로 보냄. 서버에 부하가 많음<br>
    - Next.js, Nuxt.js
    
#### CSR(Client Side Rendering)
    - 클라이언트 -> 서버 -> 서버: 소스코드 -> 클라이언트 -> 랜더링<br>
    - 클라이언트 측에서 랜더링 진행하여 서버 부하가 줄어듦. but, SEO가 약해짐. <br>
    - ex) React.js, Angular.js, Vue.js

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait

wait = WebDriverWait(driver, 10)
items = wait.until(EC.presence_of_all_elements_located(By.CSS.SELECTOR, "img.img-fluid"))